In [12]:
import datetime
import os
import subprocess
import tarfile
import MySQLdb
import pandas

In [39]:
class MysqlToCSV:
    
    __sources = ["scofcom","shagri","shfda","wanda","ygwc"]
    
    def __init__(self):
        print("Path of this program：/home/zhangtz/notebooks/PythonWorkSpace/Program")
        print("Default directory for data upload and export：/home/zhangtz/notebooks/PythonWorkSpace/Data")
        #
        print("Parameters setting for the process：")
        print("1 [SqlFile Decompression] Name of the target compression file：")
        __packagename = input()
        self.__packagename = __packagename
        print("2 [Data Import] Import to which database：")
        __importdbname = input()
        print("\n")
        self.__importdbname = __importdbname
        print("3 [Data Export(to csv)] Export from which database：")
        __exportdbname = input()
        self.__exportdbname = __exportdbname
        #
        self.__packagefullname = self.__packagename + ".tar.gz"        
        #
        self.__packagedir = "/home/zhangtz/notebooks/PythonWorkSpace/Data"
        self.__packagepath = self.__packagedir + "/" + self.__packagefullname
        self.__packagefolderdir = self.__packagedir + "/" + self.__packagename
        
    #解压sql文件       
    def sqlfileDecompression(self):
        def untar(filename, dirs):
            t = tarfile.open(filename)
            t.extractall(path = dirs)   
        try:
            untar(self.__packagepath, self.__packagefolderdir)
            print("Decompression completed. \n")
        except:
            print("Wrong path or file doesn't exist. \n")
    
    #数据导入
    def dataImport(self):
        #遍历解压路径下的sql文件
        for dirpath, dirnames, filenames in os.walk(self.__packagefolderdir):
            for filepath in filenames:
                sqlfilepath = os.path.join(dirpath, filepath)
        #source sql file
        try:
            cmd = ["mysql", "-h", "42.159.156.254", "-u", "root", "-p%s" % "inesa2014", self.__importdbname]
            process = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
            sqlsource = "source " + sqlfilepath
            output = process.communicate(bytes(sqlsource.encode(encoding="utf-8")))
            print("Source process completed. \n")
        except:
            print("Source failed. \n")
    
    #数据导出
    def dataExport(self):
        df = self.__getStatisticalData()
        self.__dataframeToCSV("s",df)
        df = self.__getHistogramData()
        self.__dataframeToCSV("h",df)

    def __getStatisticalData(self):
        #sql connection setup
        conn = MySQLdb.connect(
            host = '42.159.156.254',
            port = 3306,
            user = 'root',
            passwd = 'inesa2014',
            charset='utf8',
            db = self.__exportdbname)
        #
        cursor = conn.cursor()
        #
        key, value = [], []
        # 1 应追溯企业数
        cursor.execute("""SELECT count(*) FROM supervisedenterprise""")
        supervisedenterprisenumber = cursor.fetchone()[0]
        key.append("应追溯企业数")
        value.append(supervisedenterprisenumber)
        # 2 已注册企业数
        cursor.execute("""SELECT count(*) FROM supervisedenterprise where registered = 1""")
        registeredenterprisenumber = cursor.fetchone()[0]
        key.append("已注册企业数")
        value.append(registeredenterprisenumber)
        # 3 覆盖率
        coverrate = registeredenterprisenumber / supervisedenterprisenumber * 100
        roundedcoverrate = round(coverrate, 2)
        key.append("覆盖率")
        value.append(roundedcoverrate)
        # 4 已上传企业数
        cursor.execute("""SELECT count(*) FROM supervisedenterprise where uploaded = 1""")
        uploadedenterprisenumber = cursor.fetchone()[0]
        key.append("已上传企业数")
        value.append(uploadedenterprisenumber)
        # 5 上传率
        uploadrate = uploadedenterprisenumber / supervisedenterprisenumber * 100
        roundeduploadrate = round(uploadrate, 2)
        key.append("上传率")
        value.append(roundeduploadrate)
        # 6 可追溯产品
        cursor.execute("""SELECT count(*) FROM product""")
        supervisedproductnumber = cursor.fetchone()[0]
        key.append("可追溯产品")
        value.append(supervisedproductnumber)
        # 7 上报企业数
        for source in self.__sources:
            cursor.execute("""SELECT count(*) FROM extsysenterprise where source = %s""",(source,))
            count = cursor.fetchone()[0]
            key.append("上报企业数" + "-" + source)
            value.append(count)
        # 8 追溯数据量
        for source in self.__sources:
            cursor.execute("""SELECT sum(total) FROM extsysreportdata where source = %s""",(source,))
            sumvalue = cursor.fetchone()[0]
            key.append("追溯数据量" + "-" + source)
            value.append(sumvalue)
        #
        cursor.close()
        conn.close()
        #
        df = pandas.DataFrame([value], columns=key)
        return df
        
    def __getHistogramData(self):
        #sql connection setup
        conn = MySQLdb.connect(
            host = '42.159.156.254',
            port = 3306,
            user = 'root',
            passwd = 'inesa2014',
            charset='utf8',
            db = self.__exportdbname)
        #
        cursor = conn.cursor()
        #
        key, value = [], []
        # 查询月份列表
        monthcalender = ["01","02","03","04","05","06","07","08","09","10","11","12"]
        currentdate = datetime.date.today().strftime('%Y-%m')
        delimiter = '-'
        splitcurrentdate = currentdate.split(delimiter)
        intcurrentyear, intcurrentmonth = int(splitcurrentdate[0]), int(splitcurrentdate[1])
        datelist = []
        if (intcurrentmonth > 5):
            for m in monthcalender[intcurrentmonth - 6 : intcurrentmonth]:
                date = splitcurrentdate[0] + "-" + m
                datelist.append(date)
        else:
            n =  6 - intcurrentmonth
            for n in monthcalender[12 - n : 12]:
                date = str(intcurrentyear - 1) + "-" + n
                datelist.append(date)
            for m in monthcalender[0 : intcurrentmonth]:
                date = splitcurrentdate[0] + "-" + m
                datelist.append(date)
        #
        key.append("数据源名称")
        for date in datelist:
            key.append(date)
        #
        for source in self.__sources:
            temp = []
            temp.append(source)
            for date in datelist:
                cursor.execute("""SELECT sum(total) FROM extsysreportdata where source = %s and mon = %s""",(source,date,))
                sumvalue = cursor.fetchone()[0]
                temp.append(sumvalue)
            value.append(temp)
        #    
        cursor.close()
        conn.close()
        #
        df = pandas.DataFrame(value, columns=key)
        return df

    def __dataframeToCSV(self,sign,dataframe):
        timenow = datetime.date.today().strftime('%Y-%m-%d')
        csvfilename = timenow + "_" + self.__packagename + "_" + sign + ".csv"
        csvfilepath = self.__packagefolderdir + "/" + csvfilename
        try:
            dataframe.to_csv(csvfilepath, encoding = "utf-8", index = False)
            print("CSV exported. \n")
        except:
            print("Failed to export csv. \n")

In [33]:
a = MysqlToCSV()
a.sqlfileDecompression()
a.dataImport()
a.dataExport()

Path of this program：/home/zhangtz/notebooks/PythonWorkSpace/Program
Default directory for data upload and export：/home/zhangtz/notebooks/PythonWorkSpace/Data
Parameters setting for the process：
1 [SqlFile Decompression] Name of the target compression file：
test
2 [Data Import] Import to which database：
Test
3 [Data Export(to csv)] Export from which database：
FM2
Decompression completed. 

Source process completed. 

CSV exported. 

CSV exported. 



In [37]:
pandas.read_csv("/home/zhangtz/notebooks/PythonWorkSpace/Data/test/2018-11-19_s.csv")

In [38]:
pandas.read_csv("/home/zhangtz/notebooks/PythonWorkSpace/Data/test/2018-11-19_h.csv")